In [94]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [2]:
import timeit
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_file
import cv2
import os
from skimage import morphology
import math
import numpy as np
from tqdm import tqdm
import glob as glob
import SimpleITK as sitk
import json
import sys
from skimage.metrics import structural_similarity
import copy

In [29]:
def get_full_scan(folder_path):
    files_List = glob.glob(folder_path + '/**/*.dcm', recursive=True)
    itkimage = sitk.ReadImage(files_List[0])
    rows = int(itkimage.GetMetaData('0028|0010'))
    cols = int(itkimage.GetMetaData('0028|0011'))
    mn = 1000
    mx = 0
    for file in tqdm(files_List):
        itkimage = sitk.ReadImage(file)
        mn = np.min([mn, int(itkimage.GetMetaData('0020|0013'))])
        mx = np.max([mx, int(itkimage.GetMetaData('0020|0013'))])
    full_scan = np.ndarray(shape=(mx - mn + 1, rows, cols), dtype=float, order='F')
    new_list = np.ndarray(shape=(mx - mn + 1), dtype=object)
    for file in tqdm(files_List):
        img, n = dcm_image(file)
        n = int(n)
        full_scan[n - mn, :, :] = img[0, :, :]
        new_list[n-mn] = file
    return full_scan,new_list

def dcm_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    ins = float(itkimage.GetMetaData('0020|0013'))
    return numpyImage, ins

def get_normalized(scan,mn,mx):
    scann = copy.copy(scan)
    mn = max(mn,np.amin(scann))
    mx = min(mx,np.amax(scann))
    np.clip(scann, mn, mx, out=scann)
    d = mx - mn
    scann = (scann-mn)/d
    return scann

def dynamic_windowing(scan):
    counts,bins,bars = plt.hist(scan.flatten())
    plt.close()
    if bins[-1]>3200:
        a  = 2
        print("Type 2")
        scan = get_normalized(scan, -800, 2500)
    else:
        scan = scan
        a = 1
    return scan,a

#         counts = sorted(counts)
# #         c1 = counts[-1]/counts[-2]

#         if counts[0]<7000:
        
#             print('Type 1')
#             a=1
#             scan = get_normalized(scan,-250,1800)
#         else:
#             a=3
#             print('Type 3')
#             scan = get_normalized(scan,-250,3000)
#     return scan,a

def blur_1(image, prev = 0, s = 0):
    gray = np.zeros((image.shape[0], image.shape[1]) , dtype=int)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if image[i,j] > -300 and image[i,j] < 500:
                gray[i,j] = 1
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("org gray")
#     plt.show()
#         print(np.min(gray), np.max(gray))


    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker1)
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker2)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("gray after closing")
#     plt.show()
    gray = gray>0
    gray = morphology.remove_small_objects(gray, min_size=200)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("after removing small objects")
#     plt.show()
    gray = gray.astype(np.uint8)
    kernel = np.ones((30, 30), np.uint8)
    gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    gray = gray.astype(np.uint8)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("closing")
#     plt.show()
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = cv2.drawContours(gray, contours,-1, 255, 10)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("contours")
#     plt.show()
    gray[300:,:] = 0
#     plt.imshow(gray, cmap ='gray')
#     plt.show()
    seg = gray
    seg = np.where((seg==255),seg,0)
    z=0
    if not s==0:
        diff = np.subtract(seg,prev)
        diff = diff>1
        xx = int((diff.shape[1])/2)
        diff[:,0:(xx-100)] =0
        diff[:,(xx+100):] = 0
        diff = morphology.remove_small_objects(diff, min_size=500)
        if np.any(diff):
            for l in range(diff.shape[1]):
                for j in range(diff.shape[0]):
                    if diff[j,l]==True:
                        if np.any(prev[(j-10):(j+10),l]):
                            diff[j,l] = False
            seg = np.where((diff==True),0,seg)
            z=1

    blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
    out = np.where((seg==255), blurred_img, image)
#     plt.imshow(out,'gray')
#     plt.show()
    out = out.astype(np.int16)
#         plt.imshow(out,'gray')
#         plt.show()
    dum = copy.copy(image)
    dum[:,:] = 0
    input_skin = np.where(seg==255, image, dum)
#     plt.imshow(input_skin,'gray')
#     plt.show()
    
    dum2 = copy.copy(out)
    dum2[:,:] = 0
    output_skin = np.where(seg==255, out, dum2)
#     plt.imshow(output_skin,'gray')
#     plt.show()
    if z==0:
        return out , seg, input_skin, output_skin
    else:
        return out, prev, input_skin, output_skin

    
def blur_2(img, ds,low, high, prev = 0, s=0):
    # print("low n high: ",low, high)
    dcm = img*np.max(ds.pixel_array) #*2048#*np.max(scan)
#     plt.imshow(dcm,'gray')
#     plt.show()
    # print(np.min(dcm),np.max(dcm))
    gray = np.zeros((dcm.shape[0], dcm.shape[1]) , dtype=int)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            if dcm[i,j] > low and dcm[i,j] < high:
                gray[i,j] = 1
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("org gray")
#     plt.show()
    #         print(np.min(gray), np.max(gray))

    ker1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    ker2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker1)
    gray = cv2.morphologyEx(np.float32(gray), cv2.MORPH_CLOSE, ker2)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("gray after closing")
    #     plt.show()
    gray = gray>0
    gray = morphology.remove_small_objects(gray, min_size=200)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("after removing small objects")
    #     plt.show()
    gray = gray.astype(np.uint8)
    kernel = np.ones((30, 30), np.uint8)
    gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    gray = gray.astype(np.uint8)
    #     plt.imshow(gray , cmap = 'gray')
    #     plt.title("closing")
    #     plt.show()
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    gray = cv2.drawContours(gray, contours,-1, 255, 10)
#     plt.imshow(gray , cmap = 'gray')
#     plt.title("contours")
#     plt.show()
    gray[400:,:] = 0
#     plt.imshow(gray, cmap ='gray')
#     plt.show()

    seg = gray
    seg = np.where((seg==255),seg,0)
    z=0
    if not s==0:
        diff = np.subtract(seg,prev)
        diff = diff>1
        xx = int((diff.shape[1])/2)
        diff[:,0:(xx-100)] =0
        diff[:,(xx+100):] = 0
        diff = morphology.remove_small_objects(diff, min_size=500)
        if np.any(diff):
            for l in range(diff.shape[1]):
                for j in range(diff.shape[0]):
                    if diff[j,l]==True:
                        if np.any(prev[(j-10):(j+10),l]):
                            diff[j,l] = False
            seg = np.where((diff==True),0,seg)
            z=1

    blurred_img = cv2.GaussianBlur(ds.pixel_array, (101, 101), 800)
    
    dum = copy.copy(dcm)
    dum[:,:] = 0
    input_skin = np.where(seg==255, dcm, dum)
#     plt.imshow(input_skin,'gray')
#     plt.show()
    out = np.where((seg==255), blurred_img, ds.pixel_array)
#     plt.imshow(out,'gray')

    out = out.astype(np.int16)
    
    dum2 = copy.copy(out)
    dum2[:,:] = 0
    output_skin = np.where(seg==255, out, dum2)
#     plt.imshow(output_skin,'gray')
#     plt.show()
#     print(np.min(ds.pixel_array),np.max(ds.pixel_array))
#     ds.PixelData = out.tobytes()

#     plt.imshow(ds.pixel_array,'gray')
#     plt.title("bytes")
#     plt.show()

#     print(np.min(ds.pixel_array),np.max(ds.pixel_array))

    if z==0:
        return out , seg, input_skin, output_skin
    else:
        return out, prev, input_skin, output_skin   
    

# from skimage.metrics import structural_similarity

# def signaltonoise(a, axis=0, ddof=0):
#     a = np.asanyarray(a)
#     m = a.mean(axis)
#     sd = a.std(axis=axis, ddof=ddof)
#     return np.where(sd == 0, 0, m/sd)


def prediction_CBCT_1(scann, names, output_folder):
    output_list= []
    classUID = []
    annotations=[]

    iscan = copy.copy(scann)
    oscan = copy.copy(scann)    
    iskin = []
    oskin = []
    snr=[]
    for i in tqdm(range(scann.shape[0])):
        name = names[i]
        img = scann[i, :, :]

        dcmData = pydicom.dcmread(name)
        dicom_img = dcmData.pixel_array #for calculating parameters
        iscan[i] = dicom_img
#         plt.imshow(img,'gray')
#         plt.title("Input")
#         plt.show()
        if i == 0:
            out, prev_img, input_skin, output_skin = blur_1(img)
        else:
            out, prev_img, input_skin, output_skin = blur_1(img, prev_img, s=1)
#             print(np.min(out),np.max(out))    
#             plt.imshow(out,'gray')
#             plt.title("after ct_blur")
#             plt.show()

        out = out.astype(np.int16)
        oscan[i] = out
#             plt.imshow(out,'gray')
#             plt.title("after int16")
#             plt.show()
#             print("after blue and np16: ", np.min(out),np.max(out))

#             dcmData = pydicom.dcmread(name)
        outbytes = out.tobytes()
    
        skin_mask = input_skin
        iskin.append(skin_mask)
        After_annonymization_skin_mask = output_skin
        oskin.append(After_annonymization_skin_mask)
#             print("bytes min and maximum",np.min(outbytes.pixel_array),np.max(outbytes.pixel_array))
        dcmData.PixelData = outbytes

        # print("pixel data min and maximum",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
        # plt.imshow(dcmData.pixel_array,'gray')
        # plt.title("bytes")
        # plt.show()

        _, tail = os.path.split(name)
        des_path = os.path.join(output_folder, tail)
        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
        dcmData.save_as(des_path)

        classUID.append(str(dcmData.SOPClassUID))
        output_list.append(des_path)
        annotations.append(None)    
    
    # isnr = signaltonoise(iscan, axis=None, ddof=0)
    # osnr = signaltonoise(oscan,axis=None, ddof=0)
    # (ss, dd) = structural_similarity(iscan, oscan, full=True)

    # print(isnr, osnr, ss)
    
    return output_list, classUID, annotations, snr, iskin, oskin, iscan, oscan

def prediction_CBCT_2(scann, names, output_folder):
    print("cbct2")
    output_list= []
    classUID = []
    annotations=[]    
    
    iscan = copy.copy(scann)
    oscan = copy.copy(scann)
    snr = []
    iskin = []
    oskin = []
    
    for i in tqdm(range(scann.shape[0])):
        name = names[i]
        img = scann[i, :, :]
        low = 350
        high = 2000
        dcmData = pydicom.dcmread(name)
        if dcmData.file_meta.TransferSyntaxUID == "1.2.840.10008.1.2.4.70":
            dcmData.decompress('pylibjpeg')
            low = 250
            high = 3000
            
        dicom_img = dcmData.pixel_array
        iscan[i] = dicom_img
#         plt.imshow(img,'gray')
#         plt.title("Input")
#         plt.show()
        if i == 0:
            out, prev_img, input_skin, output_skin = blur_2(img, dcmData, low, high)
        else:
            out, prev_img, input_skin, output_skin = blur_2(img, dcmData, low, high, prev_img, s=1)
#             print(np.min(out),np.max(out))    
#             plt.imshow(out,'gray')
#             plt.title("after ct_blur")
#             plt.show()
#         skin_mask,After_annonymization_skin_mask = same_range(skin, skin_a)
        skin_mask = input_skin
        iskin.append(skin_mask)
        After_annonymization_skin_mask = output_skin
        oskin.append(After_annonymization_skin_mask)
        
        Power_of_signal = (np.sum(skin_mask)**2)
        Power_of_noise = (np.sum(After_annonymization_skin_mask)**2)
        SNR = Power_of_signal/Power_of_noise
        snr.append(SNR)
        
        out = out.astype(np.int16)
        oscan[i] = out
#             plt.imshow(out,'gray')
#             plt.title("after int16")
#             plt.show()
#             print("after blue and np16: ", np.min(out),np.max(out))

#             dcmData = pydicom.dcmread(name)
        outbytes = out.tobytes()

#             print("bytes min and maximum",np.min(outbytes.pixel_array),np.max(outbytes.pixel_array))
        dcmData.PixelData = outbytes

        # print("pixel data min and maximum",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
        # plt.imshow(dcmData.pixel_array,'gray')
        # plt.title("bytes")
        # plt.show()

        _, tail = os.path.split(name)
        des_path = os.path.join(output_folder, tail)
        dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
        dcmData.save_as(des_path)
        
#         test_ds = pydicom.dcmread(des_path)
#         test_s = test_ds.pixel_array
#         plt.imshow(test_s,'gray')
#         plt.title("read output")
#         plt.show()
        
        classUID.append(str(dcmData.SOPClassUID))
        output_list.append(des_path)
        annotations.append(None)
    
    # isnr = signaltonoise(iscan, axis=None, ddof=0)
    # osnr = signaltonoise(oscan,axis=None, ddof=0)
    # (ss, dd) = structural_similarity(iscan, oscan, full=True)

    # print(isnr, osnr, ss)
    
    return output_list, classUID, annotations, snr, iskin, oskin, iscan, oscan

def predictions_cbct(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    files_list = sorted(glob.glob((input_folder+ "/**/*.dcm"),recursive = True))
    
    initialdata = pydicom.dcmread(files_list[0])
    mode = str(initialdata.Modality)
    # start_AI = timeit.default_timer()
    if (mode=='CT') or (mode=='ct'):
        i = 0
        # start_files = timeit.default_timer()
        scan, names = get_full_scan(input_folder)
        # stop_files = timeit.default_timer()
        # print("time for reading files: ", (stop_files - start_files))
        
        inscan = copy.copy(scan)
        scann,a = dynamic_windowing(inscan)
        
        if a == 1:
            # print("Type 1, org alg")
            output_list, classUID, annotations,snr, iskin, oskin, iscan, oscan = prediction_CBCT_1(scan, names, output_folder)
        elif a == 2:
            # print("Type 2, improv alg")
            output_list, classUID, annotations,snr, iskin, oskin, iscan, oscan = prediction_CBCT_2(scann, names, output_folder)
        

#             i = i + 1
    # stop_ai = timeit.default_timer()
    # print("Time taken by AI: ", stop_ai - start_AI)
    

#     mimeType = "application/dicom"
#     mimeType = []
#     for i in range(len(output_list)):
#         mimeType.append("application/dicom")
#     image=[]
#     print("in AI",len(classUID))
#     for i in range(len(classUID)):
#         image.append(None)
#     recommendation_string = {"finding": "dummy finding","conclusion":"dummy conclusion","recommendation":"dummy recommendation"} 
#     all_result={"output_list":output_list, "classUID":classUID, "mimeType":mimeType,"annotations":annotations, "recommendation_string": recommendation_string,"image":image}
#     with open(input_folder+"/values.json", "w") as outfile:
#         json.dump(all_result, outfile)
    # print(recommendation_string)
        

    return snr, iskin, oskin, iscan, oscan#all_result

In [30]:
def same_range(imggr, imggf):
    min1 = np.min(imggr)
    max1 = np.max(imggr)

    # Find min and max of image2
    min2 = np.min(imggf)
    max2 = np.max(imggf)

    # Find overall min and max
    min_overall = min(min1, min2)
    max_overall = max(max1, max2)

    # Scale image1 to the overall range
    imggdd = (imggr - min1) / (max1 - min1) * ((max_overall - min_overall) + min_overall)

    # Scale image2 to the overall range
    imggff = (imggf - min2) / (max2 - min2) * ((max_overall - min_overall) + min_overall)
    
#     maxi = np.max(imggdd)
#     imggdd = imggdd/1024
#     imggff = imggff/1024
    return imggdd, imggff

In [95]:
d = "/home/azka/Face Anonymyzation Testing/CBCT/00505/"
l = ["/home/azka/Face Anonymyzation Testing/CBCT/00203/",
     "/home/azka/Face Anonymyzation Testing/CBCT/00262/",
     "/home/azka/Face Anonymyzation Testing/CBCT/00275/",
     "/home/azka/Face Anonymyzation Testing/CBCT/00330/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00388/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00553/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00572/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00660/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00696/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00759/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00779/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00905/",
    "/home/azka/Face Anonymyzation Testing/CBCT/00963/"]

### SSIM and PSNR

In [ ]:
from skimage.metrics import structural_similarity

In [ ]:
from math import log10, sqrt
import cv2
import numpy as np
  
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = np.max(original)#65536#np.max(np.sum(original)**2)
    psnr = 20 * log10((max_pixel-1) / sqrt(mse))
    return psnr

### Whole Scan

In [96]:
for ll in l:
    print(ll)
    ipath = ll
    snr,iskin, oskin, iscan, oscan = predictions_cbct(ipath, "/home/minha/FARZEEN/face ann/face cbct/output1/out")
#     value = PSNR(np.array(iskin), np.array(oskin))
#     print(f"PSNR value for skin unmatched is {value} dB")
#     value = PSNR(iscan, oscan)
#     print(f"PSNR value for scan unmatched is {value} dB")
#     scan_mask,After_annonymization_scan_mask = same_range(iscan, oscan)
#     value = PSNR(np.array(scan_mask ), np.array(After_annonymization_scan_mask ))
#     print(f"PSNR value for scan matched is {value} dB")
#     skin_mask,After_annonymization_skin_mask = same_range(iskin, oskin)
#     value = PSNR(np.array(skin_mask ), np.array(After_annonymization_skin_mask ))
#     print(f"PSNR value for skin matched is {value} dB")
    skin_mask,After_annonymization_skin_mask = same_range(iskin, oskin)
    score,diff = structural_similarity(np.array(skin_mask), np.array(After_annonymization_skin_mask), full=True)
    print (score)
    value = PSNR(np.array(skin_mask ), np.array(After_annonymization_skin_mask ))
    print(f"PSNR value for skin matched is {value} dB")
    print("****          ****")

/home/azka/Face Anonymyzation Testing/CBCT/00203/


  0%|                                                   | 0/460 [00:00<?, ?it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011001'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  0%|                                           | 1/460 [00:00<01:08,  6.74it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011002'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  0%|▏                                          | 2/460 [00:00<01:09,  6.55it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/value

  4%|█▋                                        | 19/460 [00:02<01:03,  6.98it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011020'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  4%|█▊                                        | 20/460 [00:02<01:02,  7.05it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011021'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  5%|█▉                                        | 21/460 [00:02<01:02,  7.08it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/value

  8%|███▌                                      | 39/460 [00:05<01:01,  6.89it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011040'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  9%|███▋                                      | 40/460 [00:05<01:00,  6.95it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011041'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
  9%|███▋                                      | 41/460 [00:05<00:59,  7.01it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/value

 13%|█████▍                                    | 59/460 [00:08<00:59,  6.79it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011060'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
 13%|█████▍                                    | 60/460 [00:08<00:58,  6.88it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/valuerep.py:290: UserWarning: Invalid value for VR UI: '1.2.392.200036.9141.281.05201.20060531090000.20161208113011061'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
 13%|█████▌                                    | 61/460 [00:08<00:57,  6.94it/s]/home/minha/anaconda3/envs/hyundai1/lib/python3.10/site-packages/pydicom/value

...

0.9474378000275094
PSNR value for skin matched is (30.47532918800226, 348711.88590754435) dB
****          ****
/home/azka/Face Anonymyzation Testing/CBCT/00963/


100%|█████████████████████████████████████████| 640/640 [00:08<00:00, 78.95it/s]


Type 2
cbct2


100%|█████████████████████████████████████████| 640/640 [05:32<00:00,  1.93it/s]


0.9402826249983974
PSNR value for skin matched is (28.722663234810796, 1451648.3456010092) dB
****          ****
